## 搭建一个分词工具

### 基于枚举方法来搭建中文分词工具

此项目需要的数据，[现代汉语常用词表.txt](https://github.com/liangqi/chinese-frequency-word-list)： 
1. 包含了中文词，当做词典来用
2. 提供了词频，相当于可以变相得到unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [59]:
# TODO: 第一步： 现代汉语常用词表.txt中读取所有中文词及对应频率。
from collections import Counter
import random
import numpy as np

with open('./data/现代汉语常用词表.txt') as f:
    lines = f.readlines()

sum = 0
word_prob = Counter()

for line in lines:
    columns = line.strip().split()
    # 重复词频率直接相加，（相同词多次出现是因为发音不同，即语义也不同，这里不做区分）
    word_prob[columns[0]] += int(columns[-1])
    sum += int(columns[-1])

# 频率转为概率
for word in word_prob:
    word_prob[word] /= sum

In [60]:
# 测试
print([{word: word_prob[word]} for word in random.sample(word_prob.keys(), 3)])
print("词典大小：%d" % len(word_prob))
print(np.sum(list(word_prob.values())))

[{'高个儿': 1.8414889771925325e-05}, {'传送带': 2.20334194331537e-05}, {'调遣': 1.4040430031872929e-05}]
词典大小：55735
1.0


In [61]:
## TODO 根据词典获得一句话的全切分，暂不考虑标点等
def sentence_break(str):
    """
    求该句话在当前词典下的全切分。
    思路：状态转移，设M[i]是从句子开始到第i个字所组成句的全切分，word是以字i结尾的可在词典中找到的词，则M[i] = M[i-len(word)] + word
    
    str: 字符串，传入的句子
    """
    
    # 存储状态
    memory = [[] for _ in range(len(str))]
    # 更新memory[0]状态
    if str[0] in word_prob:
        memory[0].append([str[0]])
    
    for i in range(1, len(str)):
        for j in range(0, i+1):
            # 从开始到当前cursor视为一个词
            if j == 0:
                if str[j:i+1] in word_prob:
                    memory[i].append([str[j:i+1]])
                continue
            # 确定依赖的之前状态存在且（达成转移条件：词存在）
            if memory[j-1] and str[j:i+1] in word_prob:
                # 状态转移过程
                for state in memory[j-1]:
                    memory[i].append(state + [str[j:i+1]])

    return memory[-1]

In [62]:
# 测试
print(sentence_break("北京的天气真好啊"))
print(sentence_break("今天的课程内容很有意思"))
print(sentence_break("经常有意见分歧"))

[['北京', '的', '天气', '真', '好', '啊'], ['北', '京', '的', '天气', '真', '好', '啊'], ['北京', '的', '天', '气', '真', '好', '啊'], ['北', '京', '的', '天', '气', '真', '好', '啊']]
[['今天', '的', '课程', '内容', '很', '有意思'], ['今', '天', '的', '课程', '内容', '很', '有意思'], ['今天', '的', '课', '程', '内容', '很', '有意思'], ['今', '天', '的', '课', '程', '内容', '很', '有意思'], ['今天', '的', '课程', '内', '容', '很', '有意思'], ['今', '天', '的', '课程', '内', '容', '很', '有意思'], ['今天', '的', '课', '程', '内', '容', '很', '有意思'], ['今', '天', '的', '课', '程', '内', '容', '很', '有意思'], ['今天', '的', '课程', '内容', '很', '有', '意思'], ['今', '天', '的', '课程', '内容', '很', '有', '意思'], ['今天', '的', '课', '程', '内容', '很', '有', '意思'], ['今', '天', '的', '课', '程', '内容', '很', '有', '意思'], ['今天', '的', '课程', '内', '容', '很', '有', '意思'], ['今', '天', '的', '课程', '内', '容', '很', '有', '意思'], ['今天', '的', '课', '程', '内', '容', '很', '有', '意思'], ['今', '天', '的', '课', '程', '内', '容', '很', '有', '意思'], ['今天', '的', '课程', '内容', '很', '有意', '思'], ['今', '天', '的', '课程', '内容', '很', '有意', '思'], ['今天', '的', '课', '程', '内容', '很', '有意', '